In [1]:
# Mohammadmilad Sayyad____801419978

# Problem 3(b): Three-hidden-layer MLP (5 -> 32 -> 16 -> 8 -> 1)
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

# ----- Load & preprocess -----
CSV_PATH = "Housing.csv"
features = ["area","bedrooms","bathrooms","stories","parking"]
target   = "price"

np.random.seed(0)
torch.manual_seed(0)

df = pd.read_csv(CSV_PATH)
X = df[features].values.astype(np.float32)
y = df[target].values.astype(np.float32).reshape(-1,1)

n = len(df)
idx = np.arange(n)
rng = np.random.default_rng(0)
rng.shuffle(idx)
cut = int(0.8*n)
tr, va = idx[:cut], idx[cut:]
Xtr, Xva = X[tr], X[va]
ytr, yva = y[tr], y[va]

# Standardize inputs
mu = Xtr.mean(axis=0)
sd = Xtr.std(axis=0); sd[sd==0]=1.0
Xtr_n = (Xtr - mu)/sd
Xva_n = (Xva - mu)/sd

Xt = torch.tensor(Xtr_n)
yt = torch.tensor(ytr)
Xv = torch.tensor(Xva_n)
yv = torch.tensor(yva)

# ----- Model -----
model = nn.Sequential(
    nn.Linear(5, 32),
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 8),
    nn.ReLU(),
    nn.Linear(8, 1)
)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# ----- Train -----
EPOCHS = 200
start_time = time.time()
for epoch in range(1, EPOCHS+1):
    model.train()
    optimizer.zero_grad()
    yhat = model(Xt)
    loss = loss_fn(yhat, yt)
    loss.backward()
    optimizer.step()
train_time = time.time() - start_time

# ----- Evaluate -----
model.eval()
with torch.no_grad():
    train_mse = loss_fn(model(Xt), yt).item()
    val_pred = model(Xv)
    val_mse = loss_fn(val_pred, yv).item()
    y_true = yv.numpy(); y_pred = val_pred.numpy()
    ss_res = ((y_true - y_pred)**2).sum()
    ss_tot = ((y_true - y_true.mean())**2).sum()
    val_r2 = 1 - ss_res/ss_tot

print("\n=== Problem 3(b) Results (Three hidden layers) ===")
print(f"Training time: {train_time:.3f} s")
print(f"Train MSE: {train_mse:,.2f}")
print(f"Val   MSE: {val_mse:,.2f}")
print(f"Val   R^2: {val_r2:.3f}")



=== Problem 3(b) Results (Three hidden layers) ===
Training time: 0.359 s
Train MSE: 26,352,642,162,688.00
Val   MSE: 25,653,269,233,664.00
Val   R^2: -6.698
